In [8]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [9]:
df = pd.read_csv('processed_dataset.csv')

In [10]:
df.head(10)

,UNIQUE_ID,GENDER,DAYS_STAY,AGE,Diagnosis_Certain Conditions originating in the Perinatal Period,"Diagnosis_Complications of Pregnancy, Childbirth, and the Puerperium",Diagnosis_Congenital Anomalies,Diagnosis_Diseases of the Blood and Blood-forming Organs,Diagnosis_Diseases of the Circulatory System,Diagnosis_Diseases of the Digestive System,...,Diagnosis_Neoplasms,Diagnosis_Supplementary Classification of External Causes of Injury and Poisoning,Diagnosis_Supplementary Classification of Factors influencing Health Status and Contact with Health Services,"Diagnosis_Symptoms, Signs and Ill-defined Conditions",Insurance_Government,Insurance_Medicaid,Insurance_Medicare,Insurance_Private,Insurance_Self Pay,avg_days
0,10000_187813,0,20,50,0,0,0,1,1,1,...,0,0,0,1,0,0,0,0,1,11
1,10001_118420,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,10
2,10002_132138,0,2,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,13
3,10003_144039,0,10,41,0,0,0,1,1,0,...,0,1,0,0,0,0,0,1,0,11
4,10004_161106,0,3,53,0,0,0,0,1,0,...,0,1,1,0,0,0,1,0,0,11
5,10004_164713,0,17,52,0,0,0,0,1,1,...,0,1,1,1,0,0,1,0,0,11
6,10005_181146,1,2,0,1,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,14
7,10006_142345,1,8,70,0,0,0,1,1,1,...,0,1,1,1,0,0,1,0,0,11
8,10007_166331,1,8,55,0,0,1,0,1,0,...,0,0,0,1,0,0,0,1,0,12
9,10008_180596,1,34,0,1,0,0,0,0,0,...,0,0,1,1,0,0,0,1,0,12


In [11]:
for col in ['AGE', 'DAYS_STAY']:
    Q1_train = df[col].quantile(0.25)
    Q3_train = df[col].quantile(0.75)
    IQR_train = Q3_train - Q1_train
    df[col] = np.where((df[col] > (Q3_train + 1.5*IQR_train)), (Q3_train + 1.5*IQR_train), df[col])

In [12]:
X = df.drop(['DAYS_STAY','UNIQUE_ID'], axis=1)
y = df['DAYS_STAY']

# divide entre treino e teste 70:30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X.head(1)

,GENDER,AGE,Diagnosis_Certain Conditions originating in the Perinatal Period,"Diagnosis_Complications of Pregnancy, Childbirth, and the Puerperium",Diagnosis_Congenital Anomalies,Diagnosis_Diseases of the Blood and Blood-forming Organs,Diagnosis_Diseases of the Circulatory System,Diagnosis_Diseases of the Digestive System,Diagnosis_Diseases of the Genitourinary System,Diagnosis_Diseases of the Musculoskeletal System and Connective Tissue,...,Diagnosis_Neoplasms,Diagnosis_Supplementary Classification of External Causes of Injury and Poisoning,Diagnosis_Supplementary Classification of Factors influencing Health Status and Contact with Health Services,"Diagnosis_Symptoms, Signs and Ill-defined Conditions",Insurance_Government,Insurance_Medicaid,Insurance_Medicare,Insurance_Private,Insurance_Self Pay,avg_days
0,0,50.0,0,0,0,1,1,1,1,1,...,0,0,0,1,0,0,0,0,1,11


In [13]:
xgb_model = XGBRegressor(objective ='reg:squarederror')
xgb_model.fit(X_train, y_train)

# Previsão
y_pred_train = xgb_model.predict(X_train)
y_pred_test = xgb_model.predict(X_test)

# Avaliação do Modelo
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

mae = mean_absolute_error(y_test, y_pred_test)

print(f'mae: {mae}')
print(f'Treino MSE: {mse_train}')
print(f'Teste MSE: {mse_test}')

mae: 4.493597867753941
Treino MSE: 27.21551300323033
Teste MSE: 34.410412647257
